<a href="https://colab.research.google.com/github/EliasEconomou/DataMining1-Sentiment-Analysis/blob/main/DataMining1_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Libraries and Reading Datafile**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import sys
import os.path
#!pip install --user -U nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [230]:
# Options
pd.options.mode.chained_assignment = None

In [231]:
# Reading data file
fileExists = os.path.exists("./drive/MyDrive/ColabNotebooks/eclass_all_with_sentiment_v2.pkl")
if fileExists:
  DF = pd.read_pickle("./drive/MyDrive/ColabNotebooks/eclass_all_with_sentiment_v2.pkl")
else:
  print('Enter data file path:')
  filePath = input()
  DF = pd.read_pickle(filePath)


# **Preprocessing and Cleaning Data**

In [232]:
# Choose a subset of the dataset's tweets
numOfRows = DF.shape[0]
percentage = 4
df = DF.iloc[:int(percentage*0.01*numOfRows)]
print("Chosen number of tweets is {}.".format(df.shape[0]))

Chosen number of tweets is 9128.


In [ ]:
df

In [234]:
# Remove punctuation, links, hashtags, new lines etc, using regular expressions
df["text"] = df["text"].replace('\.|,|\?','', regex=True)
df["text"] = df["text"].replace('\n|\t','', regex=True)
df["text"] = df["text"].replace(r'(http\S+)|(www\S+)|(#\S+)|(&\S+)|(@\S+)','', regex=True)

# Remove emojis by encoding - decoding into ascii, resulting in ignoring emojis
df["text"] = df["text"].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

# Convert text to lowercase
df["text"] = df["text"].str.lower()

# Remove stopwords like 'i','is','you', etc
stopWords = list(stopwords.words('english'))
stopWords.remove("why")
stopWords.remove("how")
stopWords.remove("against")
stopWords.remove("again")
stopWords.remove("not")
stopWords.append("'ve")
stopWords.append("'re")
# if "'d" in stopWords:
#   print(True)
# else:
#   print(False)
df["text"] = df["text"].apply(word_tokenize)
df["text"] = df["text"].apply(lambda words: [word for word in words if word not in stopWords])

# Remove any unnecessary spaces
df["text"] = df["text"].replace(' +', ' ', regex=True)


False


In [235]:
df["text"][4]

['explain', 'again', 'why', 'need', 'vaccine']

# **Analyzing Data**

In [239]:
!git reset --hard HEAD^

fatal: not a git repository (or any of the parent directories): .git
